In [1]:
from Annotation_helper import *
import numpy as np
from collections import defaultdict
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats.inter_rater import fleiss_kappa

In [2]:
import seaborn as sns
sns.set()

SMALL_SIZE = 15
MEDIUM_SIZE = 17
BIGGER_SIZE = 19

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.tight_layout()

<Figure size 432x288 with 0 Axes>

In [3]:
data_folder = "../../Data"
structured_data_folder = data_folder + "/structured_data"
text_path = data_folder + "/text"
data_path = structured_data_folder + "/test.csv"
dist_folder = structured_data_folder + "/Distances"
figure_folder = "../Figures/Results/Embeddings"
model_folder = "../Models/Embeddings"
match_folder = structured_data_folder + "/Matches/"
figure_folder = "../Figures/Results/Classic/"

In [4]:
match_files = glob.glob(match_folder + "*")

In [5]:
match_df = [pd.read_csv(mf, index_col=0).drop_duplicates(keep=False) for mf in match_files]

In [25]:
size_df = [m.shape[0] for m in match_df]
smallest_df = np.argmin(size_df)
smallest_permission_indx = match_df[smallest_df]["permission_indx"]
p_max = smallest_permission_indx.unique()[-1]

largest_indx = np.argmax(size_df)

In [7]:
def get_value_dict(df, p_max=33):
    d = defaultdict(list)
    for i in range(0, p_max):
        r = df[df['permission_indx'] == i]
        for j, row in r.iterrows():
            v = row["transaction_indx"]
            if v != "None":
                d[i].append(v)
    return d

In [17]:
d_dicts = []
for df in match_df:
    d_dicts.append(get_value_dict(df, p_max=p_max))

In [26]:
res = []
for i in range(0, p_max):
    temp_res = []
    for d in d_dicts:
        r = d[i]
        if r == []:
            temp_res.append(0)
        else:
            temp_res.append(int("".join(r)))
    res.append(temp_res)
res = np.array(res)

In [53]:
largest_res = res[:, largest_indx]
other_res = np.delete(res, largest_indx, 1)
other_size = np.delete(size_df, largest_indx, 0)

In [72]:
results_df = pd.DataFrame(index=["Cohen kappa score"])
for i in range(0, len(other_res[0])):
    cks = np.round(cohen_kappa_score(largest_res, other_res[:,i]), 2)
    size = other_size
    results_df[i] = [cks]

print(results_df.to_latex())

\begin{tabular}{lrrr}
\toprule
{} &     0 &     1 &     2 \\
\midrule
Cohen kappa score &  0.83 &  0.79 &  0.83 \\
\bottomrule
\end{tabular}



In [22]:
cohen_kappa_score(res[:,3], res[:,3])

1.0